In [2]:
import pandas as pd
#import gspread
import requests 
import io
import matplotlib.pyplot as plt
import scipy
import numpy as np
import utils
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

Part 2

In [83]:
# AL Train Data Set
ALdata = pd.read_csv('AL/AL/train', low_memory=False, lineterminator ='\n', delimiter = ' ', header = None, names = ['Word', 'Tag'])
ALdata.head(15)

,Word,Tag
0,小,B-DISTRICT
1,港,I-DISTRICT
2,四,B-POI
3,方,I-POI
4,家,I-POI
5,园,I-POI
6,17,B-HOUSENO
7,幢,I-HOUSENO
8,1363,B-CELLNO
9,室,I-CELLNO


In [84]:
# CN Train Data Set
CNdata = pd.read_csv('CN/CN/train', low_memory=False, lineterminator ='\n', delimiter = ' ', header = None, names = ['Word', 'Tag'])
CNdata.head(15)

,Word,Tag
0,这,O
1,大概,O
2,是,O
3,所有,O
4,版本,O
5,里,O
6,无忌,B-neutral
7,哥哥,I-neutral
8,对,O
9,敏敏,B-neutral


In [85]:
# EN Train Data Set
ENdata = pd.read_csv('EN/EN/train', low_memory=False, lineterminator ='\n', delimiter = ' ', header = None, names = ['Word', 'Tag'])
ENdata.head(15)

,Word,Tag
0,Municipal,B-NP
1,bonds,I-NP
2,are,B-VP
3,generally,B-ADVP
4,a,B-ADJP
5,bit,I-ADJP
6,safer,I-ADJP
7,than,B-PP
8,corporate,B-NP
9,bonds,I-NP


In [86]:
# SG Train Data Set
SGdata = pd.read_csv('SG/SG/train', low_memory=False, lineterminator ='\n', delimiter = ' ', header = None, names = ['Word', 'Tag'])
SGdata.head(15)

,Word,Tag
0,I'm,O
1,about,O
2,to,O
3,see,O
4,Adam,B-positive
5,Levine,I-positive
6,and,O
7,I'm,O
8,probably,O
9,going,O


In [87]:
print('EN:')
print(ENdata['Tag'].value_counts())
print('\n')

print('AL:')
print(ALdata['Tag'].value_counts())
print('\n')

print('CN:')
print(CNdata['Tag'].value_counts())
print('\n')

print('SG:')
print(SGdata['Tag'].value_counts())
print('\n')

EN:
I-NP       54591
B-NP       47305
O          23872
B-PP       18387
B-VP       18261
I-VP       10159
B-ADVP      3565
B-SBAR      1899
B-ADJP      1751
I-ADJP       574
B-PRT        468
I-ADVP       363
I-PP         223
I-CONJP       64
B-CONJP       49
I-SBAR        48
B-INTJ        26
B-LST         11
I-INTJ         7
I-UCP          4
B-UCP          1
Name: Tag, dtype: int64


AL:
I-POI          28472
I-ROAD         15178
I-DISTRICT     13364
I-TOWN         10748
I-CITY         10430
I-PROV          8666
B-POI           7348
B-DISTRICT      6856
B-ROAD          6324
B-CITY          5697
B-ROADNO        5061
I-ROADNO        4666
B-TOWN          4612
I-SUBPOI        4557
B-PROV          4488
B-REDUNDANT     4137
I-REDUNDANT     3668
B-HOUSENO       3455
I-COMMUNITY     3148
B-ROOMNO        3107
I-HOUSENO       3045
I-PERSON        2232
I-DEVZONE       1834
B-SUBPOI        1650
I-CELLNO        1488
B-COMMUNITY     1479
B-CELLNO        1326
I-FLOORNO       1237
B-FLOORNO       1229


In [88]:
# Write a function that estimates the emission parameters from the training set using MLE (maximum likelihood estimation)
# e(x|y) = Count (y -> x)/Count(y)
 
def emissionEstimate(data, word, tag):
    tagcheck = (data.Tag == tag)
    y = tagcheck.sum()
    xy = (data.Word[tagcheck] == word).sum()
    
    return xy/y

In [89]:
# replace the words that appear less than k times in the training set with
# special word token #UNK# before training. This leads to a "modified training set"
# if a word does not appear in the "modified training set", we replace that with #UNK# as well

#Smoothing of EN
EN = pd.DataFrame(ENdata['Word'].value_counts()<3)
EN.apply(lambda val: val == True)
k1 = EN.index[EN['Word'] == True].tolist()
ENdata.loc[ENdata['Word'].isin(k1), 'Word'] = '#UNK#'
ENdata.head(20)

,Word,Tag
0,#UNK#,B-NP
1,bonds,I-NP
2,are,B-VP
3,generally,B-ADVP
4,a,B-ADJP
5,bit,I-ADJP
6,#UNK#,I-ADJP
7,than,B-PP
8,corporate,B-NP
9,bonds,I-NP


In [91]:
#Smoothing of AL
AL = pd.DataFrame(ALdata['Word'].value_counts()<3)
AL.apply(lambda val: val == True)
k2 = AL.index[AL['Word'] == True].tolist()
ALdata.loc[ALdata['Word'].isin(k2), 'Word'] = '#UNK#'
ALdata.head(20)

,Word,Tag
0,小,B-DISTRICT
1,港,I-DISTRICT
2,四,B-POI
3,方,I-POI
4,家,I-POI
5,园,I-POI
6,17,B-HOUSENO
7,幢,I-HOUSENO
8,1363,B-CELLNO
9,室,I-CELLNO


In [92]:
#Smoothing of CN
CN = pd.DataFrame(CNdata['Word'].value_counts()<3)
CN.apply(lambda val: val == True)
k3 = CN.index[CN['Word'] == True].tolist()
CNdata.loc[CNdata['Word'].isin(k3), 'Word'] = '#UNK#'
CNdata.head(20)

,Word,Tag
0,这,O
1,大概,O
2,是,O
3,所有,O
4,版本,O
5,里,O
6,#UNK#,B-neutral
7,哥哥,I-neutral
8,对,O
9,#UNK#,B-neutral


In [93]:
#Smoothing of SG
SG = pd.DataFrame(SGdata['Word'].value_counts()<3)
SG.apply(lambda val: val == True)
k4 = SG.index[SG['Word'] == True].tolist()
SGdata.loc[SGdata['Word'].isin(k4), 'Word'] = '#UNK#'
SGdata.head(20)

,Word,Tag
0,I'm,O
1,about,O
2,to,O
3,see,O
4,Adam,B-positive
5,Levine,I-positive
6,and,O
7,I'm,O
8,probably,O
9,going,O


In [103]:
# Train



In [104]:
ALdevin = pd.read_csv('AL/AL/dev.in', low_memory=False, lineterminator ='\n', delimiter = ' ', quotechar=None, quoting=3)
CNdevin = pd.read_csv('CN/CN/dev.in', low_memory=False, lineterminator ='\n', delimiter = ' ', quotechar=None, quoting=3)
ENdevin = pd.read_csv('/Users/danial/Documents/ISTD/Machine Learning/Project/EN/dev.in', low_memory=False, lineterminator ='\n', delimiter = ' ', quotechar=None, quoting=3)
SGdevin = pd.read_csv('/Users/danial/Documents/ISTD/Machine Learning/Project/SG/dev.in', low_memory=False, lineterminator ='\n', delimiter = ' ', quotechar=None, quoting=3)
